# Upload subject sets to Zooniverse using the Panoptes client

See the [Panoptes-client documentation](https://panoptes-python-client.readthedocs.io/en/v1.1/index.html#).

In [5]:
import os
import glob
import re
import io
from PIL import Image
from panoptes_client import Panoptes, Project, SubjectSet, Subject
from credentials import ZOONIVERSE_USER, ZOONIVERSE_PW

In [6]:
Panoptes.connect(username=ZOONIVERSE_USER, password=ZOONIVERSE_PW)

In [7]:
project = Project.find('8678')
project.title

'Sydney Stock Exchange'

## Create a new subject set

In [8]:
# New subject set
subject_set = SubjectSet()

subject_set.links.project = project
subject_set.display_name = 'Assorted headers'

subject_set.save()

{'subject_sets': [{'id': '77733',
   'display_name': 'Assorted headers',
   'set_member_subjects_count': 0,
   'metadata': {},
   'created_at': '2019-09-04T02:29:13.466Z',
   'updated_at': '2019-09-04T02:29:13.466Z',
   'href': '/subject_sets/77733',
   'links': {'project': '8678', 'workflows': [], 'subjects': []}}],
 'links': {'subject_sets.project': {'href': '/projects/{subject_sets.project}',
   'type': 'projects'},
  'subject_sets.workflows': {'href': '/workflows?subject_set_id={subject_sets.id}',
   'type': 'workflows'}},
 'meta': {'subject_sets': {'page': 1,
   'page_size': 20,
   'count': 1,
   'include': [],
   'page_count': 1,
   'previous_page': None,
   'next_page': None,
   'first_href': '/subject_sets',
   'previous_href': None,
   'next_href': None,
   'last_href': '/subject_sets'}}}

## Get and existing subject set

In [52]:
subject_set = SubjectSet.find('76903')
subject_set.display_name

'1929 headers'

## Upload subjects to a subject set

Generate a list of items from a directory of images. Each item has the following fields:

* `image_path` – full path to image
* `image_name` – image file name
* `volume` – volume identifier
* `volume_number` – numeric
* `page_number` – numeric

In [9]:
file_pattern = '/Volumes/Workspace/mycode/stock-exchange/notebooks/processed/samples/headers/*header.jpg'
image_files = glob.glob(file_pattern)
data = []
for image_file in image_files:
    image_path = image_file
    image_name = image_file.split('/')[-1]
    volume = re.search(r'(N193-\d+)_', image_name).group(1)
    volume_number = int(volume.split('-')[1])
    page_number = int(re.search(r'N193-\d+_(\d+)', image_name).group(1))
    data.append({'image_path': image_path, 'image_name': image_name, 'volume': volume, 'volume_number': volume_number, 'page_number': page_number})

In [10]:
# View the results
data

[{'image_path': '/Volumes/Workspace/mycode/stock-exchange/notebooks/processed/samples/headers/N193-121_0038-header.jpg',
  'image_name': 'N193-121_0038-header.jpg',
  'volume': 'N193-121',
  'volume_number': 121,
  'page_number': 38},
 {'image_path': '/Volumes/Workspace/mycode/stock-exchange/notebooks/processed/samples/headers/N193-083_0117-header.jpg',
  'image_name': 'N193-083_0117-header.jpg',
  'volume': 'N193-083',
  'volume_number': 83,
  'page_number': 117},
 {'image_path': '/Volumes/Workspace/mycode/stock-exchange/notebooks/processed/samples/headers/N193-103_0317-header.jpg',
  'image_name': 'N193-103_0317-header.jpg',
  'volume': 'N193-103',
  'volume_number': 103,
  'page_number': 317},
 {'image_path': '/Volumes/Workspace/mycode/stock-exchange/notebooks/processed/samples/headers/N193-016_0172-header.jpg',
  'image_name': 'N193-016_0172-header.jpg',
  'volume': 'N193-016',
  'volume_number': 16,
  'page_number': 172},
 {'image_path': '/Volumes/Workspace/mycode/stock-exchange/n

Loop through the metadata creating new Zooniverse subjects. The images are resized before they are added to the subjects.

In [11]:
new_subjects = []
for item in data:
    # New Zooniverse subject
    subject = Subject()
    
    # Link subject to project
    subject.links.project = project
    
    # Open the image
    img = Image.open(item['image_path'])
    w, h = img.size
    
    # Get the resize ratio based on a maximum width of 2000 px
    ratio = 2000 / w
    
    # Resize image using the ratio
    img = img.resize((int(w*ratio), int(h*ratio)), Image.LANCZOS)
    
    # Create a bytes object and save the image to it
    b = io.BytesIO()
    img.save(b, 'JPEG')
    b.seek(0)
    
    # Add the bytes object to the subject
    subject.add_location(b)
    
    # Add metadata to the subject
    subject.metadata['image'] = item['image_name']
    subject.metadata['volume'] = item['volume']
    subject.metadata['volume_number'] = item['volume_number']
    subject.metadata['page_number'] = item['page_number']
    subject.metadata['year'] = 1930
    
    # Save the subject
    subject.save()
    
    # Add to the list of subjects
    new_subjects.append(subject)

In [12]:
# Save the list of subjects to the subject set
subject_set.add(new_subjects)